### import libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import geopandas as gpd
from shapely.geometry import Point

### load wildfires dataset

In [2]:
db_file_15 = "cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/wildfires/FPA_FOD_20170508.sqlite"

con = sqlite3.connect(db_file_15)
df_wild = pd.read_sql_query("SELECT * from Fires", con, index_col='OBJECTID')
con.close()

df_wild

,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,1,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,13,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,27,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,43,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,44,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,591814,...,A,40.481637,-122.389375,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x9...
1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,569419,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,574245,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...


### process dates

In [3]:
def weekday_from_date(day, month, year):
    return datetime.date(day=day, month=month, year=year).weekday()

df_wild["DISCO_DATE"] = pd.to_datetime(df_wild["FIRE_YEAR"]*1000 + df_wild["DISCOVERY_DOY"], format = "%Y%j")
df_wild[["DISCOVERY_DATE", "FIRE_YEAR", "DISCOVERY_DOY", "DISCO_DATE"]].head()

,DISCOVERY_DATE,FIRE_YEAR,DISCOVERY_DOY,DISCO_DATE
OBJECTID,,,,
1,2453403.5,2005,33,2005-02-02
2,2453137.5,2004,133,2004-05-12
3,2453156.5,2004,152,2004-05-31
4,2453184.5,2004,180,2004-06-28
5,2453184.5,2004,180,2004-06-28


### select columns of interest

In [4]:
df_interest = df_wild[["FOD_ID",
                       "FPA_ID",
                       "FIRE_YEAR",
                       "DISCO_DATE",
                       "STAT_CAUSE_CODE",
                       "STAT_CAUSE_DESCR",
                       "LATITUDE",
                       "LONGITUDE",
                       "STATE",
                       #"COUNTY",
                       #"FIPS_CODE",
                       #"FIPS_NAME"
                      ]]
df_interest

,FOD_ID,FPA_ID,FIRE_YEAR,DISCO_DATE,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE
OBJECTID,,,,,,,,,
1,1,FS-1418826,2005,2005-02-02,9.0,Miscellaneous,40.036944,-121.005833,CA
2,2,FS-1418827,2004,2004-05-12,1.0,Lightning,38.933056,-120.404444,CA
3,3,FS-1418835,2004,2004-05-31,5.0,Debris Burning,38.984167,-120.735556,CA
4,4,FS-1418845,2004,2004-06-28,1.0,Lightning,38.559167,-119.913333,CA
5,5,FS-1418847,2004,2004-06-28,1.0,Lightning,38.559167,-119.933056,CA
...,...,...,...,...,...,...,...,...,...
1880461,300348363,2015CAIRS29019636,2015,2015-09-26,13.0,Missing/Undefined,40.481637,-122.389375,CA
1880462,300348373,2015CAIRS29217935,2015,2015-10-05,9.0,Miscellaneous,37.617619,-120.938570,CA
1880463,300348375,2015CAIRS28364460,2015,2015-05-02,13.0,Missing/Undefined,37.617619,-120.938570,CA


### calculate wildfires geometry points

In [5]:
gdf_wildfires = gpd.GeoDataFrame(df_interest, geometry=gpd.points_from_xy(df_interest['LONGITUDE'], df_interest['LATITUDE']), crs="EPSG:4269")
gdf_wildfires

,FOD_ID,FPA_ID,FIRE_YEAR,DISCO_DATE,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,geometry
OBJECTID,,,,,,,,,,
1,1,FS-1418826,2005,2005-02-02,9.0,Miscellaneous,40.036944,-121.005833,CA,POINT (-121.00583 40.03694)
2,2,FS-1418827,2004,2004-05-12,1.0,Lightning,38.933056,-120.404444,CA,POINT (-120.40444 38.93306)
3,3,FS-1418835,2004,2004-05-31,5.0,Debris Burning,38.984167,-120.735556,CA,POINT (-120.73556 38.98417)
4,4,FS-1418845,2004,2004-06-28,1.0,Lightning,38.559167,-119.913333,CA,POINT (-119.91333 38.55917)
5,5,FS-1418847,2004,2004-06-28,1.0,Lightning,38.559167,-119.933056,CA,POINT (-119.93306 38.55917)
...,...,...,...,...,...,...,...,...,...,...
1880461,300348363,2015CAIRS29019636,2015,2015-09-26,13.0,Missing/Undefined,40.481637,-122.389375,CA,POINT (-122.38938 40.48164)
1880462,300348373,2015CAIRS29217935,2015,2015-10-05,9.0,Miscellaneous,37.617619,-120.938570,CA,POINT (-120.93857 37.61762)
1880463,300348375,2015CAIRS28364460,2015,2015-05-02,13.0,Missing/Undefined,37.617619,-120.938570,CA,POINT (-120.93857 37.61762)


### import extended (100 feet) forests shapes

In [6]:
ext_forest_file = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/temp/S_USA.ProclaimedForest/us_forests_ext_d100.shp'

gdf_forest_ext = gpd.read_file(ext_forest_file)
gdf_forest_ext

ERROR 1: PROJ: proj_create_from_database: Open of /cloud-home/XLOUIS/.magellan/conda/envs/geo/share/proj failed


,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry
0,295435010328,Allegheny National Forest,740686.211,0.323984,3.965097,"MULTIPOLYGON (((-79.47447 41.54646, -79.47447 ..."
1,295380010328,Angeles National Forest,695598.432,0.276018,4.674530,"MULTIPOLYGON (((-118.39144 34.29308, -118.3923..."
2,295458010328,Angelina National Forest,398093.002,0.152499,3.691482,"MULTIPOLYGON (((-94.52605 31.25947, -94.52605 ..."
3,295372010328,Apache National Forest,1869703.247,0.736196,6.526785,"POLYGON ((-109.65438 34.13032, -109.65614 34.1..."
4,295400010328,Apalachicola National Forest,633591.300,0.240186,3.367179,"MULTIPOLYGON (((-85.10001 30.05484, -85.10021 ..."
...,...,...,...,...,...,...
149,295401010328,White River National Forest,2482446.167,1.052907,13.329421,"MULTIPOLYGON (((-108.12547 39.26733, -108.1254..."
150,109214010328,Whitman National Forest,1311082.638,0.605272,18.320346,"MULTIPOLYGON (((-117.72385 45.26576, -117.7286..."
151,295436010328,Willamette National Forest,1794216.105,0.816252,8.953350,"MULTIPOLYGON (((-122.62455 43.76543, -122.6247..."
152,295365010328,William B. Bankhead National Forest,349259.985,0.138292,2.451515,"MULTIPOLYGON (((-87.52237 34.42819, -87.52147 ..."


### import state shapes and joining with forests

In [7]:
states_file = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/states/tl_2023_us_state.shp'

gdf_states = gpd.read_file(states_file)
gdf_states = gdf_states.set_crs("EPSG:4269")
#gdf_states

In [8]:
gdf_forest = gpd.sjoin(gdf_forest_ext, gdf_states[['NAME', 'geometry']], how='left').drop('index_right', axis=1)
gdf_forest = gdf_forest.groupby(by=gdf_forest_ext.columns.to_list())['NAME'].apply(lambda x: ','.join(x)).reset_index()
gdf_forest = gdf_forest.rename(columns={'NAME':'STATE'})

out_path = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/wildfires/us_forests_ext_d100+state.csv.gz'
gdf_forest.to_csv(out_path, compression='gzip')
gdf_forest

,PROCLAIMED,FORESTNAME,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,geometry,STATE
0,105935010328,Kaibab National Forest,1601002.978,0.647156,9.140002,"MULTIPOLYGON (((-112.39468 36.11971, -112.3919...",Arizona
1,106640010328,Fremont National Forest,1713917.383,0.760518,11.282002,"MULTIPOLYGON (((-120.21961 42.44060, -120.2196...","California,Oregon"
2,106887010328,Mt. Baker National Forest,1317676.621,0.648567,10.659633,"MULTIPOLYGON (((-121.01830 48.99886, -121.0185...",Washington
3,107266010328,Olympic National Forest,695868.519,0.337426,8.179740,"MULTIPOLYGON (((-123.95161 47.34539, -123.9516...",Washington
4,107474010328,Wallowa National Forest,1064857.162,0.496222,9.027734,"MULTIPOLYGON (((-117.69431 45.45237, -117.6943...","Oregon,Idaho,Washington"
...,...,...,...,...,...,...,...
149,295505010328,Trinity National Forest,1188515.052,0.512099,6.547120,"MULTIPOLYGON (((-123.55983 40.70216, -123.5598...",California
150,66329010328,Bighorn National Forest,1112645.655,0.509899,4.943363,"POLYGON ((-107.94566 44.92726, -107.94566 44.9...","Wyoming,Montana"
151,93007010328,Gifford Pinchot National Forest,1532172.644,0.722290,11.679336,"MULTIPOLYGON (((-122.36441 46.28825, -122.3639...","Oregon,Washington"
152,96812010328,Manti-La Sal National Forest,1337654.214,0.561732,9.953308,"MULTIPOLYGON (((-109.19589 38.49227, -109.2002...","Colorado,Utah"


### join forest ids to wildfires

In [9]:
joined_gdf = gpd.sjoin(gdf_wildfires, gdf_forest_ext[['PROCLAIMED', 'FORESTNAME', 'geometry']], how='left')
joined_gdf = joined_gdf.loc[joined_gdf['PROCLAIMED'].notnull()].drop('index_right', axis=1)

# Créer une nouvelle colonne CAUSES_NAT_HUM basée sur la valeur de STAT_CAUSE_CODE
joined_gdf['CAUSES_NAT_HUM'] = joined_gdf['STAT_CAUSE_CODE'].apply(lambda x: 0 if x == 1.0 else 1)
joined_gdf

,FOD_ID,FPA_ID,FIRE_YEAR,DISCO_DATE,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,geometry,PROCLAIMED,FORESTNAME,CAUSES_NAT_HUM
OBJECTID,,,,,,,,,,,,,
1,1,FS-1418826,2005,2005-02-02,9.0,Miscellaneous,40.036944,-121.005833,CA,POINT (-121.00583 40.03694),295390010328,Plumas National Forest,1
2,2,FS-1418827,2004,2004-05-12,1.0,Lightning,38.933056,-120.404444,CA,POINT (-120.40444 38.93306),295382010328,Eldorado National Forest,0
3,3,FS-1418835,2004,2004-05-31,5.0,Debris Burning,38.984167,-120.735556,CA,POINT (-120.73556 38.98417),295382010328,Eldorado National Forest,1
4,4,FS-1418845,2004,2004-06-28,1.0,Lightning,38.559167,-119.913333,CA,POINT (-119.91333 38.55917),295382010328,Eldorado National Forest,0
5,5,FS-1418847,2004,2004-06-28,1.0,Lightning,38.559167,-119.933056,CA,POINT (-119.93306 38.55917),295382010328,Eldorado National Forest,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880428,300348172,2013CAIRS22549226,2013,2013-04-20,8.0,Children,34.633333,-118.966666,CA,POINT (-118.96667 34.63333),295387010328,Los Padres National Forest,1
1880435,300348212,2015CAIRS28442302,2015,2015-08-06,1.0,Lightning,39.655817,-121.234567,CA,POINT (-121.23457 39.65582),295390010328,Plumas National Forest,0
1880450,300348289,2015CAIRS28493610,2015,2015-08-27,13.0,Missing/Undefined,34.666666,-119.333333,CA,POINT (-119.33333 34.66667),295387010328,Los Padres National Forest,1


### export dataframe

In [10]:
# export weather data
out_path = 'cloud-data/digitalrnd-projects-ireland/Vaccine/Research/Magellan/users/XLO/test/wildfires/wildfires_data_forest.csv.gz'
joined_gdf.to_csv(out_path, compression='gzip')

print('file_ok')

file_ok
